In [8]:
import sys
sys.path.append('..')

import os
import json
import re

import pandas as pd

In [9]:
datasets_path = os.path.join('..', 'datasets')

In [10]:
# Big color dataset
dpath_big = os.path.join(datasets_path, 'color-dict-big.json')
with open(dpath_big, 'r') as x:
    big = json.load(x)

In [11]:
# Standardize big color dataset
big_colors = pd.io.json.json_normalize(list(big['colors']))
big_colors = big_colors.drop(columns=['hex', 'luminance'])
big_colors = big_colors.rename(columns={'rgb.r': 'r', 'rgb.g': 'g', 'rgb.b': 'b'})

In [12]:
# Small color dataset
dpath_small = os.path.join(datasets_path, 'color-dict-small.json')
with open(dpath_small, 'r') as x:
    small = json.load(x)

In [13]:
# Standardize small color dataset
small_colors = []
for colorPage in small['colorPage']:
    for entry in colorPage['colorEntry']:
        small_colors.append(entry)
small_colors = pd.io.json.json_normalize(list(small_colors))
small_colors = small_colors.rename(columns={
    'colorName': 'name', 'RGB8.red': 'r', 
    'RGB8.green': 'g', 'RGB8.blue': 'b'})
small_colors['name'] = small_colors['name'].apply(lambda n: re.sub('\(SW.*\)', '', n).strip())

In [56]:
# Small Colors EDA
small_colors['num_words'] = small_colors['name'].apply(lambda c: len(c.split()))
small_colors['num_words'].value_counts()

2    982
1    473
3     69
4      2
Name: num_words, dtype: int64

In [58]:
# Some long color names
small_colors[small_colors['num_words'] >= 3].head()

,b,g,r,name,num_words
14,209,196,139,After the Rain,3
71,133,153,158,At Ease Soldier,3
88,180,162,131,Baby Blue Eyes,3
89,138,185,187,Baby Bok Choy,3
138,57,54,50,Black of Night,3


In [59]:
# Big colors EDA
big_colors['num_words'] = big_colors['name'].apply(lambda c: len(c.split()))
big_colors['num_words'].value_counts()

2    12795
1     3728
3     1422
4      125
5       10
6        1
Name: num_words, dtype: int64

In [61]:
# Some long color names
big_colors[big_colors['num_words'] >= 4].head()

,name,b,g,r,num_words
2,20000 Leagues Under the Sea,112,25,25,5
7,A Certain Shade Of Green,238,237,209,5
8,A Dime a Dozen,228,221,211,4
9,A Hint of Incremental Blue,137,103,69,5
10,A Pair of Brown Eyes,146,175,191,5
